# Import Libraries

In [208]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from pandas_profiling import ProfileReport
import numpy as np
from sklearn.preprocessing import LabelEncoder , StandardScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn. ensemble import GradientBoostingRegressor,RandomForestRegressor

# Loading Data

In [209]:
df = pd.read_csv('Train.csv')

In [210]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,4235.4750
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,604.4969
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2358.5261
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,811.6203
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,736.0226


In [211]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.250109
std,4.643456,0.051598,62.275067,8.371760,1485.314935
min,4.555000,0.000000,31.290000,1985.000000,-70.343796
25%,8.773750,0.026989,93.826500,1987.000000,1020.596825
50%,12.600000,0.053931,143.012800,1999.000000,1973.155200
75%,16.850000,0.094585,185.643700,2004.000000,2995.172000
max,21.350000,0.328391,266.888400,2009.000000,11394.745000


In [212]:
df.shape

(8523, 12)

In [213]:
#ProfileReport(df)

# Drop Columns

In [214]:
df.drop(columns = ['Item_Identifier'],inplace=True)

In [215]:
df.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,4235.4750
1,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,604.4969
2,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2358.5261
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,811.6203
4,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,736.0226


# Fill Null Values

In [216]:
df['Item_Weight'] = df['Item_Weight'].fillna(df['Item_Weight'].mean())

In [217]:
df['Outlet_Size'].unique()

array(['Medium', nan, 'High', 'Small'], dtype=object)

In [218]:
df = df.fillna({'Outlet_Size':'Medium'})

# Replace Values

In [219]:
df = df.replace({'Item_Fat_Content' : {'low fat':'Low Fat' ,'LF' : 'Low Fat' ,'reg':'Regular'}})

# Label Encoding

In [220]:
lb = LabelEncoder()

In [221]:
outlet_size = df['Outlet_Size'].unique()
df['Outlet_Size'] = lb.fit_transform(df['Outlet_Size'])
outlet_size_num = df.Outlet_Size.unique()
outlet_size_df = pd.DataFrame(data = {'outlet_size': outlet_size ,'outlet_size_num':outlet_size_num})
outlet_size_df

,outlet_size,outlet_size_num
0,Medium,1
1,High,0
2,Small,2


In [222]:
item_fat = df.Item_Fat_Content.unique()
df['Item_Fat_Content'] = lb.fit_transform(df['Item_Fat_Content'])
item_fat_num = df.Item_Fat_Content.unique()
item_fat_df = pd.DataFrame(data = {'item_fat': item_fat ,'item_fat_num':item_fat_num})
item_fat_df

,item_fat,item_fat_num
0,Low Fat,0
1,Regular,1


In [223]:
Outlet_Location = df['Outlet_Location_Type'].unique()
df['Outlet_Location_Type'] = lb.fit_transform(df['Outlet_Location_Type'])
Outlet_Location_num = df['Outlet_Location_Type'].unique()
Outlet_Location_df = pd.DataFrame(data = {'Outlet_Location': Outlet_Location ,'Outlet_Location_num':Outlet_Location_num})
Outlet_Location_df

,Outlet_Location,Outlet_Location_num
0,Tier 1,0
1,Tier 3,2
2,Tier 2,1


In [224]:
Outlet_Type = df['Outlet_Type'].unique()
df['Outlet_Type'] = lb.fit_transform(df['Outlet_Type'])
Outlet_Type_num = df['Outlet_Type'].unique()
Outlet_Type_df = pd.DataFrame(data = {'Outlet_Type': Outlet_Type ,'Outlet_Type_num':Outlet_Type_num})
Outlet_Type_df

,Outlet_Type,Outlet_Type_num
0,Supermarket Type1,1
1,Supermarket Type2,2
2,Grocery Store,0
3,Supermarket Type3,3


In [225]:
Item_Type = df['Item_Type'].unique()
df['Item_Type'] = lb.fit_transform(df['Item_Type'])
Item_Type_num = df['Item_Type'].unique()
Item_Type_df = pd.DataFrame(data = {'Item_Type': Item_Type ,'Item_Type_num':Item_Type_num})
Item_Type_df

,Item_Type,Item_Type_num
0,Dairy,4
1,Soft Drinks,14
2,Meat,10
3,Fruits and Vegetables,6
4,Household,9
5,Baking Goods,0
6,Snack Foods,13
7,Frozen Foods,5
8,Breakfast,2
9,Health and Hygiene,8


In [226]:
Outlet_Identifier = df['Outlet_Identifier'].unique()
df['Outlet_Identifier'] = lb.fit_transform(df['Outlet_Identifier'])
Outlet_Identifier_num = df['Outlet_Identifier'].unique()
Outlet_Identifier_df = pd.DataFrame(data = {'Outlet_Identifier': Outlet_Identifier ,'Outlet_Identifier_num':Outlet_Identifier_num})
Outlet_Identifier_df

,Outlet_Identifier,Outlet_Identifier_num
0,OUT049,9
1,OUT018,3
2,OUT010,0
3,OUT013,1
4,OUT027,5
5,OUT045,7
6,OUT017,2
7,OUT046,8
8,OUT035,6
9,OUT019,4


In [227]:
df.dtypes

Item_Weight                  float64
Item_Fat_Content               int32
Item_Visibility              float64
Item_Type                      int32
Item_MRP                     float64
Outlet_Identifier              int32
Outlet_Establishment_Year      int64
Outlet_Size                    int32
Outlet_Location_Type           int32
Outlet_Type                    int32
Item_Outlet_Sales            float64
dtype: object

In [228]:
x = df.drop(columns = 'Item_Outlet_Sales' )

In [229]:
x

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.300,0,0.016047,4,249.8092,9,1999,1,0,1
1,5.920,1,0.019278,14,48.2692,3,2009,1,2,2
2,17.500,0,0.016760,10,141.6180,9,1999,1,0,1
3,19.200,1,0.000000,6,182.0950,0,1998,1,2,0
4,8.930,0,0.000000,9,53.8614,1,1987,0,2,1
...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0,0.056783,13,214.5218,1,1987,0,2,1
8519,8.380,1,0.046982,0,108.1570,7,2002,1,1,1
8520,10.600,0,0.035186,8,85.1224,6,2004,2,1,1
8521,7.210,1,0.145221,13,103.1332,3,2009,1,2,2


In [230]:
y = df['Item_Outlet_Sales']

In [231]:
y

0       4235.4750
1        604.4969
2       2358.5261
3        811.6203
4        736.0226
          ...    
8518    2955.1711
8519    1634.5360
8520    1241.0330
8521    1696.4160
8522    1014.8792
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [233]:
x1 = df.drop(columns = "Item_Outlet_Sales")

# Standard scalling


In [234]:
scaler = StandardScaler()

In [235]:
x_scalar = scaler.fit_transform(x1)

In [236]:
x_scalar

array([[-0.84187169, -0.73814723, -0.97073217, ..., -0.28458121,
        -1.36933384, -0.25265831],
       [-1.64170589,  1.35474328, -0.90811123, ..., -0.28458121,
         1.09156913,  1.00297245],
       [ 1.09855449, -0.73814723, -0.95691733, ..., -0.28458121,
        -1.36933384, -0.25265831],
       ...,
       [-0.53424315, -0.73814723, -0.59978449, ...,  1.38127431,
        -0.13888236, -0.25265831],
       [-1.33644372,  1.35474328,  1.53287976, ..., -0.28458121,
         1.09156913,  1.00297245],
       [ 0.45963367, -0.73814723, -0.41193591, ...,  1.38127431,
        -1.36933384, -0.25265831]])

In [237]:
x_df = pd.DataFrame(x_scalar)

In [238]:
x_df

,0,1,2,3,4,5,6,7,8,9
0,-0.841872,-0.738147,-0.970732,-0.766479,1.747454,1.507813,0.139541,-0.284581,-1.369334,-0.252658
1,-1.641706,1.354743,-0.908111,1.608963,-1.489023,-0.607071,1.334103,-0.284581,1.091569,1.002972
2,1.098554,-0.738147,-0.956917,0.658786,0.010040,1.507813,0.139541,-0.284581,-1.369334,-0.252658
3,1.500838,1.354743,-1.281758,-0.291391,0.660050,-1.664513,0.020085,-0.284581,1.091569,-1.508289
4,-0.929428,-0.738147,-1.281758,0.421242,-1.399220,-1.312032,-1.293934,-1.950437,1.091569,-0.252658
...,...,...,...,...,...,...,...,...,...,...
8518,-1.418084,-0.738147,-0.181193,1.371418,1.180783,-1.312032,-1.293934,-1.950437,1.091569,-0.252658
8519,-1.059578,1.354743,-0.371154,-1.716656,-0.527301,0.802852,0.497909,-0.284581,-0.138882,-0.252658
8520,-0.534243,-0.738147,-0.599784,0.183698,-0.897208,0.450371,0.736822,1.381274,-0.138882,-0.252658
8521,-1.336444,1.354743,1.532880,1.371418,-0.607977,-0.607071,1.334103,-0.284581,1.091569,1.002972


In [295]:
x_train , x_test ,y_train , y_test = train_test_split(x_scalar , y , test_size = .10,random_state =  250)

# Models fitting

# Linear Regression

In [296]:
from sklearn.linear_model import LinearRegression

In [297]:
model = LinearRegression()

In [298]:
model.fit(x_train, y_train)

LinearRegression()

In [299]:
model.score(x_train,y_train)*100

67.13299887896203

In [300]:
model.score(x_test, y_test)*100

69.53009803017632

In [301]:
y_test_pred = model.predict(x_test)

In [302]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(y_test, y_test_pred), 2))

Mean absolute error = 601.01
Mean squared error = 646903.45
Median absolute error = 472.47
Explain variance score = 0.7
R2 score = 0.7


## XgBoost 

In [303]:
xgb_model_1 = xgb.XGBRegressor(learning_rate = .01, n_estimators = 400)

In [304]:
xgb_model_1.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=400, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [305]:
xgb_model_1.score(x_train,y_train)

0.8385486206294918

In [306]:
xgb_model_1.score(x_test,y_test)

0.8332926485737466

In [294]:
y_test_pred = xgb_model_1.predict(x_test)

In [280]:
y_test_pred

array([3116.4783  , 6193.6694  ,  613.4025  , 1697.7711  , 2125.5286  ,
       2382.0051  ,  292.78546 , 1842.6506  ,  851.34344 , 1652.1437  ,
       4637.949   ,  261.99585 ,  343.87076 ,  854.8293  ,  776.3687  ,
       4369.8975  , 1871.5466  ,  797.0273  , 2355.2764  , 2033.5482  ,
       3782.0334  , 3226.5393  , 3274.6191  , 3813.8596  , 1382.854   ,
       1625.7565  ,  654.94507 ,  372.55942 ,  618.2142  , 3594.451   ,
        948.2759  , 3047.3374  , 6136.3887  , 1714.1307  , 1286.2754  ,
        573.3441  , 4790.196   , 3800.522   , 1515.3381  , 3719.4065  ,
       3648.2563  , 1532.498   , 3452.394   , 2617.0015  , 3486.3975  ,
       2352.4924  ,  526.1061  , 1335.748   , 3207.8657  ,  649.36804 ,
       2536.3733  , 3513.2754  , 2590.5425  , 2792.622   , 2692.9675  ,
       3877.4822  , 1353.7858  , 4391.0254  , 1642.4553  , 3146.8464  ,
       1647.3577  ,  477.5167  , 2446.0952  ,  302.97156 ,  123.766106,
        569.3236  ,  656.5585  ,  309.30078 , 3036.8584  , 1089.

In [281]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(y_test, y_test_pred), 2))

Mean absolute error = 389.4
Mean squared error = 353934.71
Median absolute error = 207.22
Explain variance score = 0.83
R2 score = 0.83


## Gradient Boost

In [307]:
 gb_parameters = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
                         'n_estimators':[100,250,500,750,1000,1250,1500,1750],
                         'max_depth':[2,3,4,5,6,7],
                         'min_samples_split':[2,4,6,8,10,20,40,60,100],
                         'min_samples_leaf':[1,3,5,7,9],
                         'max_features':[2,3,4,5,6,7],
                         'subsample':[0.7,0.75,0.8,0.85,0.9,0.95,1]
                        }

In [308]:
gb_reg = GradientBoostingRegressor()
random_gb = RandomizedSearchCV(estimator=gb_reg,
                                        param_distributions=gb_parameters,
                                        cv=5,
                                        scoring='neg_root_mean_squared_error',
                                        n_iter=10,
                                        n_jobs=-1,
                                        verbose=2,
                                        random_state=101
                                        )

In [309]:
random_gb.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.15, 0.1, 0.05, 0.01,
                                                          0.005, 0.001],
                                        'max_depth': [2, 3, 4, 5, 6, 7],
                                        'max_features': [2, 3, 4, 5, 6, 7],
                                        'min_samples_leaf': [1, 3, 5, 7, 9],
                                        'min_samples_split': [2, 4, 6, 8, 10,
                                                              20, 40, 60, 100],
                                        'n_estimators': [100, 250, 500, 750,
                                                         1000, 1250, 1500,
                                                         1750],
                                        'subsample': [0.7, 0.75, 0.8, 0.85, 0.9,
                                                      0.95, 1]},
                   random

In [310]:
 best_param = random_gb.best_params_

In [311]:
gb_model = GradientBoostingRegressor(subsample = best_param['subsample'],
                                                n_estimators = best_param['n_estimators'],
                                                min_samples_split = best_param['min_samples_split'],
                                                min_samples_leaf = best_param['min_samples_leaf'],
                                                max_features = best_param['max_features'],
                                                max_depth = best_param['max_depth'],
                                                learning_rate = best_param['learning_rate'])

In [312]:
gb_model.fit(x_train,y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=6, max_features=5,
                          min_samples_leaf=5, min_samples_split=20,
                          n_estimators=750, subsample=0.85)

In [313]:
gb_model.score(x_train,y_train)

0.9468557421535003

In [314]:
gb_model.score(x_test,y_test)

0.8427103739021591

In [318]:
y_test_pred = gb_model.predict(x_test)

In [319]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(y_test, y_test_pred), 2))

Mean absolute error = 378.49
Mean squared error = 333940.04
Median absolute error = 218.07
Explain variance score = 0.84
R2 score = 0.84


## Random Forest

In [52]:
 rf_parameters={'n_estimators': [100,200,300,400,500,600],
                       'max_features': ['auto', 'sqrt'],
                       'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
                       'min_samples_split': [2, 5, 10],
                       'min_samples_leaf': [1, 2, 4],
                       'bootstrap': [True, False]
                      }

In [53]:
rf_reg = RandomForestRegressor()
random_rf = RandomizedSearchCV(estimator=rf_reg,
                                        param_distributions=rf_parameters,
                                        cv=5,
                                        scoring='neg_root_mean_squared_error',
                                        n_iter=10,
                                        n_jobs=-1,
                                        verbose=2,
                                        random_state=101
                                        )

In [54]:
random_rf.fit(x_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600]},
                   random_state=101, scoring='neg_root_mean_squared_error',
                   verbose=2)

In [55]:
best_param = random_rf.best_params_

In [56]:
rf_model = RandomForestRegressor(n_estimators = best_param['n_estimators'],
                                            min_samples_split = best_param['min_samples_split'],
                                            min_samples_leaf = best_param['min_samples_leaf'],
                                            max_features = best_param['max_features'],
                                            max_depth = best_param['max_depth'],
                                            bootstrap = best_param['bootstrap'])

In [57]:
rf_model.fit(x_train,y_train)

RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=2,
                      n_estimators=400)

In [58]:
rf_model.score(x_train,y_train)

0.682132839229031

In [59]:
rf_model.score(x_test,y_test)

0.6232745632936549

In [ ]:
y_test_pred = rf_model.predict(x_test)

In [ ]:
import sklearn.metrics as sm
print("Mean absolute error =", round(sm.mean_absolute_error(y_test, y_test_pred), 2)) 
print("Mean squared error =", round(sm.mean_squared_error(y_test, y_test_pred), 2)) 
print("Median absolute error =", round(sm.median_absolute_error(y_test, y_test_pred), 2)) 
print("Explain variance score =", round(sm.explained_variance_score(y_test, y_test_pred), 2)) 
print("R2 score =", round(sm.r2_score(y_test, y_test_pred), 2))

In [320]:
pickle.dump(xgb_model_1, open("BigMart_Prediction.pkl", "wb"))

In [ ]:
pickle.dump(scaler, open("Standardscaler.pkl", "wb"))